c:\Python312\python.exe


In [18]:
%pip install --user --upgrade langchain langchain-experimental langchain-openai python-dotenv pyvis %pip wikipedia


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '%pip': Expected package name at the start of dependency specifier
    %pip
    ^


In [ ]:
from dotenv import load_dotenv
import os
import sys

# Print the Python executable path
print(sys.executable)

# Load the .env file
load_dotenv()
# Get API key from environment variable 
# (make sure the key is present in .env file in the project directory)
api_key = os.getenv("OPENAI_API_KEY")

In [36]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-4o",)

graph_transformer = LLMGraphTransformer(llm=llm)

In [34]:
# Importing the Wikipedia library
import wikipedia

# Read Wikipedia page for Jean-Paul Sartre
sartre_summary = wikipedia.page("Jean-Paul Sartre").content

with open("wikipedia_sartre.txt", "w", encoding="utf-8") as f:
    f.write(sartre_summary)

print(sartre_summary[:500]) 

# Read Wikipedia page for Albert Camus
pageids = wikipedia.search("Albert Camus")
if not pageids:
    raise ValueError("No page found for Albert Camus")
# Albert Camus Page can not be retreivd !
camus_summary = wikipedia.page(pageid=983).content

with open("wikipedia_camus.txt", "w", encoding="utf-8") as f:
    f.write(camus_summary)

print(camus_summary[:500])


Jean-Paul Charles Aymard Sartre (, US also ; French: [saʁtʁ]; 21 June 1905 – 15 April 1980) was a French philosopher, playwright, novelist, screenwriter, political activist, biographer, and literary critic, considered a leading figure in 20th-century French philosophy and Marxism. Sartre was one of the key figures in the philosophy of existentialism (and phenomenology). His work has influenced sociology, critical theory, post-colonial theory, and literary studies. He was awarded the 1964 Nobel P
Albert Camus ( ka-MOO; French: [albɛʁ kamy] ; 7 November 1913 – 4 January 1960) was a French philosopher, author, dramatist, journalist, world federalist, and political activist. He was the recipient of the 1957 Nobel Prize in Literature at the age of 44, the second-youngest recipient in history. His works include The Stranger, The Plague, The Myth of Sisyphus, The Fall and The Rebel.
Camus was born in French Algeria to pied-noir parents. He spent his childhood in a poor neighbourhood and later

In [37]:
documents = [Document(page_content=camus_summary)]
graph_documents = await graph_transformer.aconvert_to_graph_documents(documents)

In [42]:
from pprint import pprint

# For displaying nodes
print("NODES:")
pprint([{"id": node.id, "type": node.type, "properties": node.properties} for node in graph_documents[0].nodes])

# For displaying relationships
print("\nRELATIONSHIPS:")
pprint([{
    "source": rel.source.id,
    "type": rel.type,
    "target": rel.target.id
} for rel in graph_documents[0].relationships])

NODES:
[{'id': 'Albert Camus', 'properties': {}, 'type': 'Person'},
 {'id': 'Catherine Hélène Camus', 'properties': {}, 'type': 'Person'},
 {'id': 'Lucien Camus', 'properties': {}, 'type': 'Person'},
 {'id': 'Louis Germain', 'properties': {}, 'type': 'Person'},
 {'id': 'Jean Grenier', 'properties': {}, 'type': 'Person'},
 {'id': 'Simone Hié', 'properties': {}, 'type': 'Person'},
 {'id': 'Francine Faure', 'properties': {}, 'type': 'Person'},
 {'id': 'Jean-Paul Sartre', 'properties': {}, 'type': 'Person'},
 {'id': 'María Casares', 'properties': {}, 'type': 'Person'},
 {'id': 'Michel Gallimard', 'properties': {}, 'type': 'Person'},
 {'id': 'Nobel Prize In Literature', 'properties': {}, 'type': 'Award'},
 {'id': 'The Stranger', 'properties': {}, 'type': 'Literary work'},
 {'id': 'The Plague', 'properties': {}, 'type': 'Literary work'},
 {'id': 'The Myth Of Sisyphus', 'properties': {}, 'type': 'Literary work'},
 {'id': 'The Fall', 'properties': {}, 'type': 'Literary work'},
 {'id': 'The Reb

In [38]:
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Albert Camus', type='Person', properties={}), Node(id='Catherine Hélène Camus', type='Person', properties={}), Node(id='Lucien Camus', type='Person', properties={}), Node(id='Louis Germain', type='Person', properties={}), Node(id='Jean Grenier', type='Person', properties={}), Node(id='Simone Hié', type='Person', properties={}), Node(id='Francine Faure', type='Person', properties={}), Node(id='Jean-Paul Sartre', type='Person', properties={}), Node(id='María Casares', type='Person', properties={}), Node(id='Michel Gallimard', type='Person', properties={}), Node(id='Nobel Prize In Literature', type='Award', properties={}), Node(id='The Stranger', type='Literary work', properties={}), Node(id='The Plague', type='Literary work', properties={}), Node(id='The Myth Of Sisyphus', type='Literary work', properties={}), Node(id='The Fall', type='Literary work', properties={}), Node(id='The Rebel', type='Literary work', properties={}), Node(id='Absurdism', type='Philosophy', propert

In [39]:
from pyvis.network import Network

def visualize_graph(graph_documents, output_file="knowledge_graph.html"):
    # Create network
    net = Network(height="1200px", width="100%", directed=True,
                      notebook=False, bgcolor="#222222", font_color="white")
    
    nodes = graph_documents[0].nodes
    relationships = graph_documents[0].relationships

    # Build lookup for valid nodes
    node_dict = {node.id: node for node in nodes}
    
    # Filter out invalid edges and collect valid node IDs
    valid_edges = []
    valid_node_ids = set()
    for rel in relationships:
        if rel.source.id in node_dict and rel.target.id in node_dict:
            valid_edges.append(rel)
            valid_node_ids.update([rel.source.id, rel.target.id])


    # Track which nodes are part of any relationship
    connected_node_ids = set()
    for rel in relationships:
        connected_node_ids.add(rel.source.id)
        connected_node_ids.add(rel.target.id)

    # Add valid nodes
    for node_id in valid_node_ids:
        node = node_dict[node_id]
        try:
            net.add_node(node.id, label=node.id, title=node.type, group=node.type)
        except:
            continue  # skip if error

    # Add valid edges
    for rel in valid_edges:
        try:
            net.add_edge(rel.source.id, rel.target.id, label=rel.type.lower())
        except:
            continue  # skip if error

    # Configure physics
    net.set_options("""
            {
                "physics": {
                    "forceAtlas2Based": {
                        "gravitationalConstant": -100,
                        "centralGravity": 0.01,
                        "springLength": 200,
                        "springConstant": 0.08
                    },
                    "minVelocity": 0.75,
                    "solver": "forceAtlas2Based"
                }
            }
            """)
        
    net.save_graph(output_file)
    print(f"Graph saved to {os.path.abspath(output_file)}")

    # Try to open in browser
    try:
        import webbrowser
        webbrowser.open(f"file://{os.path.abspath(output_file)}")
    except:
        print("Could not open browser automatically")
        
# Run the function
visualize_graph(graph_documents)

Graph saved to d:\Users\makbar\OneDrive - Rayonnance Technologies\0.Rayonnance\0.AI\knowledge-graph\knowledge_graph.html


In [40]:
allowed_nodes = ["Person", "Location", "Award", "BornIn", "Organization", "ResearchField"]
allowed_relationships = [
    ("Person", "WORKS_AT", "Organization"),
    ("Person", "SPOUSE", "Person"),
    ("Person", "AWARD", "Award"),
    ("Organization", "IN_LOCATION", "Location"),
    ("Person", "FIELD_OF_RESEARCH", "ResearchField")
]
graph_transformer_rel_defined = LLMGraphTransformer(
  llm=llm,
  allowed_nodes=allowed_nodes,
  allowed_relationships=allowed_relationships
)
graph_documents_rel_defined = await graph_transformer_rel_defined.aconvert_to_graph_documents(documents)

In [41]:
# Visualize graph
visualize_graph(graph_documents_rel_defined, output_file="knowledge_graph_restricted.html")

Graph saved to d:\Users\makbar\OneDrive - Rayonnance Technologies\0.Rayonnance\0.AI\knowledge-graph\knowledge_graph_restricted.html
